In [3]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import metrics
from read_tfrecord import *
from mtcnn_model import Rnet,cls_ohem,cal_accuracy,bbox_ohem
from tqdm import tqdm

In [4]:
data_path = "data/24/train_RNet_landmark.tfrecord_shuffle"

In [5]:
def load_pokemon(mode='train'):
    """ 加载pokemon数据集的工具！
    :param root:    数据集存储的目录
    :param mode:    mode:当前加载的数据是train,val,还是test
    :return:
    """
    # # 创建数字编码表,范围0-4;
    # name2label = {}  # "sq...":0   类别名:类标签;  字典 可以看一下目录,一共有5个文件夹,5个类别：0-4范围;
    # for name in sorted(os.listdir(os.path.join(root))):     # 列出所有目录;
    #     if not os.path.isdir(os.path.join(root, name)):
    #         continue
    #     # 给每个类别编码一个数字
    #     name2label[name] = len(name2label.keys())
 
    # 读取Label信息;保存索引文件images.csv
    # [file1,file2,], 对应的标签[3,1] 2个一一对应的list对象。
    # 根据目录,把每个照片的路径提取出来,以及每个照片路径所对应的类别都存储起来，存储到CSV文件中。
    size = 24
    images,labels,boxes = red_tf(data_path,size)
 
    # 图片切割成，训练70%，验证15%，测试15%。
    if mode == 'train':                                                     # 100% 训练集
        images = images[:int(len(images))]
        labels = labels[:int(len(labels))]
        boxes  = boxes[:int(len(boxes))]
    elif mode == 'val':                                                     # 15% = 70%->85%  验证集
        images = images[int(0.7 * len(images)):int(0.85 * len(images))]
        labels = labels[int(0.7 * len(labels)):int(0.85 * len(labels))]
        boxes = boxes[int(0.7 * len(boxes)):int(0.85 * len(boxes))]
    else:                                                                   # 15% = 70%->85%  测试集
        images = images[int(0.85 * len(images)):]
        labels = labels[int(0.85 * len(labels)):]
        boxes = boxes[int(0.85 * len(boxes)):]
    ima = tf.data.Dataset.from_tensor_slices(images)
    lab = tf.data.Dataset.from_tensor_slices(labels)
    roi = tf.data.Dataset.from_tensor_slices(boxes)
 
    train_data = tf.data.Dataset.zip((ima, lab, roi)).shuffle(1000).batch(16)
    train_data = list(train_data.as_numpy_iterator())
    return train_data

# 图像色相变换
def image_color_distort(inputs):
    inputs = tf.image.random_contrast(inputs, lower=0.5, upper=1.5)
    inputs = tf.image.random_brightness(inputs, max_delta=0.2)
    inputs = tf.image.random_hue(inputs,max_delta= 0.2)
    inputs = tf.image.random_saturation(inputs,lower = 0.5, upper= 1.5)
    return inputs

In [8]:
def train(epoch):
    model = Rnet()
    #model.load_weights("rnet.h5")
 
    optimizer = keras.optimizers.Adam(learning_rate=1e-3)
    off = 1000
    acc_meter = metrics.Accuracy()
    for epoch in tqdm(range(eopch)):
 
        for i,(img,lab,boxes) in enumerate(load_pokemon("train")):
 
 
            img = image_color_distort(img)
            # 开一个gradient tape, 计算梯度
            with tf.GradientTape() as tape:
                cls_prob, bbox_pred = model(img)
                cls_loss = cls_ohem(cls_prob, lab)
                bbox_loss = bbox_ohem(bbox_pred, boxes,lab)
                # landmark_loss = landmark_loss_fn(landmark_pred, landmark_batch, label_batch)
                # accuracy = cal_accuracy(cls_prob, label_batch)
 
 
                total_loss_value = cls_loss + 0.5 * bbox_loss
                grads = tape.gradient(total_loss_value, model.trainable_variables)
                optimizer.apply_gradients(zip(grads, model.trainable_variables))
            if i % 200 == 0:
                print('Training loss (for one batch) at step %s: %s' % (i, float(total_loss_value)))
                print('Seen so far: %s samples' % ((i + 1) * 16))
 
 
        for i, (v_img, v_lab1, boxes) in enumerate(load_pokemon("val")):
            v_img = image_color_distort(v_img)
            with tf.GradientTape() as tape:
                cls_prob, bbox_pred = model(v_img)
                cls_loss = cls_ohem(cls_prob, v_lab1)
                bbox_loss = bbox_ohem(bbox_pred, boxes,v_lab1)
                # landmark_loss = landmark_loss_fn(landmark_pred, landmark_batch, label_batch)
                # accuracy = cal_accuracy(cls_prob, label_batch)
 
 
                total_loss_value = cls_loss + 0.5 * bbox_loss
                grads = tape.gradient(total_loss_value, model.trainable_variables)
                optimizer.apply_gradients(zip(grads, model.trainable_variables))
            if i % 200 == 0:
                print('val___ loss (for one batch) at step %s: %s' % (i, float(total_loss_value)))
                print('Seen so far: %s samples' % ((i + 1) * 16))
    model.save_weights('./Weights/Rnet_wight/rnet_30.ckpt')

In [9]:
train(30)

  0%|          | 0/30 [00:00<?, ?it/s]

<MapDataset shapes: {image/encoded: (), image/label: (), image/roi: (4,)}, types: {image/encoded: tf.string, image/label: tf.int64, image/roi: tf.float32}>
Training loss (for one batch) at step 0: 0.7099941968917847
Seen so far: 6 samples
Training loss (for one batch) at step 200: 0.759408175945282
Seen so far: 1206 samples
Training loss (for one batch) at step 400: 0.36534103751182556
Seen so far: 2406 samples
<MapDataset shapes: {image/encoded: (), image/label: (), image/roi: (4,)}, types: {image/encoded: tf.string, image/label: tf.int64, image/roi: tf.float32}>
val___ loss (for one batch) at step 0: 0.48153743147850037
Seen so far: 6 samples


  3%|▎         | 1/30 [00:30<14:39, 30.34s/it]

<MapDataset shapes: {image/encoded: (), image/label: (), image/roi: (4,)}, types: {image/encoded: tf.string, image/label: tf.int64, image/roi: tf.float32}>
Training loss (for one batch) at step 0: 0.379951536655426
Seen so far: 6 samples
Training loss (for one batch) at step 200: 0.6552779078483582
Seen so far: 1206 samples
Training loss (for one batch) at step 400: 0.45284023880958557
Seen so far: 2406 samples
<MapDataset shapes: {image/encoded: (), image/label: (), image/roi: (4,)}, types: {image/encoded: tf.string, image/label: tf.int64, image/roi: tf.float32}>
val___ loss (for one batch) at step 0: 0.5992808938026428
Seen so far: 6 samples


  7%|▋         | 2/30 [01:01<14:19, 30.69s/it]

...

 93%|█████████▎| 28/30 [14:33<01:00, 30.48s/it]

<MapDataset shapes: {image/encoded: (), image/label: (), image/roi: (4,)}, types: {image/encoded: tf.string, image/label: tf.int64, image/roi: tf.float32}>
Training loss (for one batch) at step 0: 0.07273033261299133
Seen so far: 6 samples
Training loss (for one batch) at step 200: 0.12310439348220825
Seen so far: 1206 samples
Training loss (for one batch) at step 400: 0.014622936956584454
Seen so far: 2406 samples
<MapDataset shapes: {image/encoded: (), image/label: (), image/roi: (4,)}, types: {image/encoded: tf.string, image/label: tf.int64, image/roi: tf.float32}>
val___ loss (for one batch) at step 0: 0.017498411238193512
Seen so far: 6 samples


 97%|█████████▋| 29/30 [15:03<00:30, 30.45s/it]

<MapDataset shapes: {image/encoded: (), image/label: (), image/roi: (4,)}, types: {image/encoded: tf.string, image/label: tf.int64, image/roi: tf.float32}>
Training loss (for one batch) at step 0: 0.11741043627262115
Seen so far: 6 samples
Training loss (for one batch) at step 200: 0.07665160298347473
Seen so far: 1206 samples
Training loss (for one batch) at step 400: 0.028185630217194557
Seen so far: 2406 samples
<MapDataset shapes: {image/encoded: (), image/label: (), image/roi: (4,)}, types: {image/encoded: tf.string, image/label: tf.int64, image/roi: tf.float32}>
val___ loss (for one batch) at step 0: 0.004512411076575518
Seen so far: 6 samples


100%|██████████| 30/30 [15:34<00:00, 31.15s/it]
